In [1]:
!pip install webdriver-manager

In [2]:
from selenium import webdriver
import datetime
import time
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
#Open a new web browser.
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/ricardolu/.wdm/drivers/chromedriver/mac64/87.0.4280.88]


In [4]:
#Open the web page that we want open and log in.
driver.get('https://products.wholefoodsmarket.com/search?sort=relevance&store=10408&category=dairy-eggs')
time.sleep(5)

In [5]:
#scroll down the page to the bottom
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [6]:
#Grab the webpage as a BeautifulSoup object
soup = BeautifulSoup(driver.page_source,'html.parser')

In [7]:
#Obtain Product Names and append to blank list
clean_product_names = []
product_name = []
product_names = soup.find_all('div',class_='ProductCard-Name--1o2Gg')

for i in range(len(product_names)):
    each_name = product_names[i].text.strip()
    product_name.append(each_name)
    clean_product_names.append(each_name.split(",")[0])

In [8]:
#Obtain Product Prices and append to blank list (this will accidentally disguise sales prices as regular prices)
clean_prices = []
prices = soup.find_all('div',class_='Row__15gM6 ProductCard-PriceRow--g7T92')

for i in range(len(prices)):
    each_price = prices[i].text.strip().split("$")
    if each_price[0] == '':
        each_price = each_price[1]
    else:
        each_price = each_price[0]
    clean_prices.append(each_price)

In [9]:
# clean_starting_prices = dirty_prices - clean_prices
dirty_prices = []
for i in range(len(prices)):
    dirty_prices.append(prices[i].text.strip())
    
# to obtain the starting price if it has a sale
clean_starting_prices = []

for i in range(len(dirty_prices)):
    clean_starting_price = dirty_prices[i].replace(clean_prices[i],'')
    clean_starting_prices.append(clean_starting_price[1:])

In [10]:
#Obtain product quantities and append to blank list
clean_quantities = []
for i in range(len(clean_prices)):
    if '/lb' in str(clean_prices[i]) or '1 lb' in str(product_name[i]):
        clean_quantities.append('1 LB')
    elif '1 each' in str(product_name[i]):
        clean_quantities.append('1 EA')
    elif "," in str(product_name[i]):
        if len(product_name[i].split(",")[1]) < 10:
            clean_quantities.append(product_name[i].split(",")[1])
        else:
            clean_quantities.append(None)
#    elif 'lb' or 'oz' in str(product_name[i]):
#        clean_quantities.append(product_name[i])
    else:
        clean_quantities.append(None)

In [11]:
# data cleaning: remove $, /lb and convert cents to dollars
def clean_price(prices):
    clean_price = []
    for price in prices:
        price = str(price)
        if price == '':
            clean_price.append(price)
        else:
            price = price.replace('$','')
            price = price.replace('/lb','')
            if '¢' in price:
                price = price.replace('¢','')
                price = float(price)/100
            clean_price.append(float(price))
    return clean_price
    
clean_prices = clean_price(clean_prices)
clean_starting_prices = clean_price(clean_starting_prices)

In [12]:
#Obtain whether or not there was a sale and append to blank list
clean_sale_yes_or_no = []
big_HTML_container_1 = soup.find_all('div',class_='ProductCard-Root--3g5WI')

# Looking for string "On Sale"
for i in range(len(big_HTML_container_1)):
    if 'On Sale' in str(big_HTML_container_1[i]):
        clean_sale_yes_or_no.append('Sale')
    else:
        clean_sale_yes_or_no.append('')

In [13]:
#make a product classification column
product_type = []
product = "Dairy"
for i in range(len(clean_product_names)):
    product_type.append(product)

In [14]:
#make a column to generate the date the data was pulled
dates = []
todays_date = datetime.date.today()
for i in range(len(clean_product_names)):
    dates.append(todays_date)

In [15]:
#make a column to generate the store name where the data is from
store = []
for i in range(len(clean_product_names)):
    store.append('Whole Foods')
    

In [16]:
#Create and outsheet dataframe
Whole_Foods_Dairy_Data = {'Date': dates,'Store':store,'Product Type':product_type, 'Product Name': clean_product_names,'Price':clean_prices,'Quantity Sold In':clean_quantities,'Sale on This Product?':clean_sale_yes_or_no,'If Sale, Original Price':clean_starting_prices}
Whole_Foods_Dairy_Dataframe = pd.DataFrame(data=Whole_Foods_Dairy_Data)


Whole_Foods_Dairy_Dataframe.to_csv ('Whole Foods '+product+' '+str(todays_date)+'.csv', index = False, header=True)

In [17]:
#Close the webpage
driver.close()

**More Resources**

Selenium with Python: https://selenium-python.readthedocs.io/

Selenium Python tutorial: https://www.geeksforgeeks.org/selenium-python-tutorial/